In [1]:
import os
os.environ['WANDB_DISABLED']="true"

# Install and load libraries

In [2]:
!pip install -q bitsandbytes datasets accelerate loralib rouge_score evaluate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 28.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 59.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

In [3]:
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
!nvidia-smi -L

GPU 0: NVIDIA L4 (UUID: GPU-babd2073-d1df-f111-2861-d924c289a337)


In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

from datasets import Dataset, load_metric

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# set up the model with Lora

Code reference for setting up Lora: https://colab.research.google.com/drive/14xo6sj4dARk8lXZbOifHEn1f_70qNAwy?usp=sharing#scrollTo=4iwHGzKBN6wk

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb

model_name = "allenai/led-base-16384"
led_base_lora = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    load_in_8bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
for param in led_base_lora.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

led_base_lora.gradient_checkpointing_enable()  # reduce number of stored activations
led_base_lora.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
led_base_lora.lm_head = CastOutputToFloat(led_base_lora.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="Seq2Seq" # set this for CLM or Seq2Seq
)

led_base_lora = get_peft_model(led_base_lora, config)
print_trainable_parameters(led_base_lora)

trainable params: 589824 || all params: 162434304 || trainable%: 0.36311541680259857


In [ ]:
!nvidia-smi

Sat Jul 13 23:49:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              47W / 400W |    729MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Load data

In [ ]:
# Paths to the files
path_train_long = '/content/drive/MyDrive/W266_Project/Data/train_long.csv'
path_test_long = '/content/drive/MyDrive/W266_Project/Data/test_long.csv'
path_val_long = '/content/drive/MyDrive/W266_Project/Data/val_long.csv'

# Load the datasets
train_long = pd.read_csv(path_train_long)
test_long = pd.read_csv(path_test_long)
val_long = pd.read_csv(path_val_long)

# Check the first few rows to ensure they're loaded correctly
print("long_dialogue_train_data:")
print(train_long.head())

# Check the shape
print("Shape of long_dialogue_train_data:")
print(train_long.shape)
print("Shape of long_dialogue_test_data:")
print(test_long.shape)
print("Shape of long_dialogue_val_data:")
print(val_long.shape)


long_dialogue_train_data:
                                            dialogue  \
0  doctor donna torres , date of birth , 08/01/19...   
1  doctor: Good morning, Mr. patient. I'm Dr. doc...   
2  doctor: Hello Mrs. patient, thank you for comi...   
3  doctor hi virginia how're you today patient i'...   
4  doctor: Hello, Mrs. patient, welcome back. How...   

                                                note  Dialogue_Length  \
0  SUBJECTIVE CHIEF COMPLAINT Annual health maint...             1867   
1  SUBJECTIVE CHIEF COMPLAINT Patient reports fru...              330   
2  SUBJECTIVE CHIEF COMPLAINT Left arm pain after...              654   
3  SUBJECTIVE CHIEF COMPLAINT Right knee pain. HI...             1367   
4  SUBJECTIVE CHIEF COMPLAINT Recurrent low back ...              320   

   Note_Length  
0          471  
1          249  
2          452  
3          375  
4          257  
Shape of long_dialogue_train_data:
(1102, 4)
Shape of long_dialogue_test_data:
(180, 4)
Shape of

In [ ]:
train_long.head(1)

,dialogue,note,Dialogue_Length,Note_Length
0,"doctor donna torres , date of birth , 08/01/19...",SUBJECTIVE CHIEF COMPLAINT Annual health maint...,1867,471


The model can take 16384 tokens, based on data analysis:
- Min dialogue length: 135
- Ave dialogue length: 426
- Max dialogue length: 3050
- 95% dialogue length: 966
- median dialogue length: 353
- Min summary length: 83
- Ave summary length: 276
- Max summary length: 752
- 95% summary length: 458
- Median summary length: 257

In [ ]:
train_long = Dataset.from_pandas(train_long)
test_long = Dataset.from_pandas(test_long)
val_long = Dataset.from_pandas(val_long)


# Fine-tuning the model

## Tokenization

Code reference for tokenization: https://colab.research.google.com/drive/12LjJazBl7Gam0XBPy_y0CTOJZeZ34c2v?usp=sharing

In [ ]:
# tokennize
max_input_length = 2048
max_output_length = 1024
def process_data_to_model_inputs(batch):
    start_prompt = 'Summarize the following patientdoctor dialogue into 2 sectios:"SUBJECTIVE CHIEF COMPLAINT" and "ASSESSMENT AND PLAN". Include all medically relevant information, including family history, diagnosis, past medical and surgical history, immunizations, lab results and known allergies.Dialogue:.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in batch["dialogue"]]
    # tokenize the inputs and labels
    inputs = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["note"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
# map the function
batch_size = 2
train_long = train_long.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "note", "dialogue","Dialogue_Length","Note_Length"],
)

Map:   0%|          | 0/1102 [00:00<?, ? examples/s]

In [ ]:
test_long = test_long.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "note", "dialogue","Dialogue_Length","Note_Length"],
)

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

In [ ]:
val_long = val_long.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "note", "dialogue","Dialogue_Length","Note_Length"],
)

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

In [ ]:
from random import sample
# check the shape after tokenization
sample_input_id = test_long['input_ids'][0:1]
sample_label = test_long['labels'][0:1]

print("tokenized_train_exp: ", test_long)
print("\nsample_input_id: ", len(sample_input_id[0]), sample_input_id)
print("\nsample_label: ", len(sample_label[0]), sample_label)


tokenized_train_exp:  Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 180
})

sample_input_id:  2048 [[0, 38182, 3916, 2072, 5, 511, 3186, 44914, 6054, 88, 132, 26299, 4544, 36856, 104, 12027, 33302, 10002, 3858, 7720, 597, 15823, 43362, 17831, 113, 8, 22, 17042, 12147, 12613, 4248, 37330, 845, 39682, 70, 25749, 4249, 335, 6, 217, 284, 750, 6, 9726, 6, 375, 1131, 8, 15535, 750, 6, 13998, 18391, 6, 6348, 775, 8, 684, 26331, 4, 48201, 35, 4, 50118, 50118, 44914, 35, 2497, 662, 6, 427, 4, 83, 4, 3837, 47, 13, 567, 11, 452, 4, 38, 1346, 47, 214, 259, 13, 10, 92, 3186, 10437, 4, 3186, 35, 280, 18, 235, 4, 3299, 35, 2615, 47, 1137, 162, 10, 410, 828, 59, 110, 1131, 750, 116, 3186, 35, 2647, 6, 38, 348, 56, 80, 15948, 11, 5, 375, 4, 20, 78, 21, 11, 550, 9, 3503, 6, 8, 5, 200, 21, 11, 830, 9, 3010, 4, 572, 14, 6, 38, 554, 519, 22107, 8, 23385, 1588, 5347, 354, 4, 38, 348, 57, 11, 8, 66, 9, 5, 1098, 6, 8, 38, 2226, 230, 4, 25871, 4, 38, 

## Training

In [ ]:

import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import DataCollatorForSeq2Seq

# ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=led_base_lora,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.optim import AdamW

# Define custom optimizer
optimizer = AdamW(led_base_lora.parameters(), lr=3e-5, weight_decay=0.01)

# Define the output directory
#repository_id = "./output_directory"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    #logging & evaluation strategies
    #logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=led_base_lora,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_long,
    eval_dataset=val_long,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),  # Pass the custom optimizer
)

In [ ]:
# Start training
with torch.cuda.amp.autocast():
    trainer.train()


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.275301,2.975800,0.029900,2.025700,2.962300,20.000000


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1252: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
!nvidia-smi

Sun Jul 14 00:24:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              47W / 400W |   5579MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# save the model
trainer.save_model("./led_base_finetuned_dialogue_G_ACI")
tokenizer.save_pretrained("./led_base_finetuned_dialogue_G_ACI")

('./led_base_finetuned_dialogue_G_ACI/tokenizer_config.json',
 './led_base_finetuned_dialogue_G_ACI/special_tokens_map.json',
 './led_base_finetuned_dialogue_G_ACI/vocab.json',
 './led_base_finetuned_dialogue_G_ACI/merges.txt',
 './led_base_finetuned_dialogue_G_ACI/added_tokens.json',
 './led_base_finetuned_dialogue_G_ACI/tokenizer.json')

In [ ]:
# push to huggingface
from huggingface_hub import HfApi, HfFolder

model_name = "litt5/led_base_finetuned_dialogue_G_ACI"
api = HfApi()

# Create the repository if it doesn't exist
api.create_repo(repo_id=model_name, repo_type="model", exist_ok=True)

# Upload the model files
api.upload_folder(
    folder_path="./led_base_finetuned_dialogue_G_ACI",
    repo_id=model_name,
    repo_type="model"
)

adapter_model.safetensors:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/litt5/led_base_finetuned_dialogue_G_ACI/commit/d2079d2fa08fd8195ccfffdfe3822189850de263', commit_message='Upload folder using huggingface_hub', commit_description='', oid='d2079d2fa08fd8195ccfffdfe3822189850de263', pr_url=None, pr_revision=None, pr_num=None)

# Model evaluation

In [7]:

# Load the fine-tuned model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("litt5/led_base_finetuned_dialogue_G_ACI")
model = AutoModelForSeq2SeqLM.from_pretrained("litt5/led_base_finetuned_dialogue_G_ACI", torch_dtype=torch.bfloat16)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

adapter_config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

In [18]:
# Paths to the files
path_test_long = '/content/drive/MyDrive/W266_Project/Data/test_long.csv'


# Load the datasets

test_long = pd.read_csv(path_test_long)



# Check the shape

print("Shape of long_dialogue_test_data:")
print(test_long.shape)



Shape of long_dialogue_test_data:
(180, 4)


In [21]:
subset_size = int(0.3 * len(test_long))
sub_set_test = test_long.sample(n=subset_size, random_state=42)  # random_state for reproducibility
len(sub_set_test)

54

In [22]:
sub_set_test.head()

,dialogue,note,Dialogue_Length,Note_Length
19,"doctor: Good morning, Mrs. patient's name. Tha...",SUBJECTIVE CHIEF COMPLAINT Confusion and histo...,418,464
42,"doctor: Hello, patient. I understand you're he...",SUBJECTIVE CHIEF COMPLAINT Foot pain. HISTORY ...,418,328
153,doctor today i'm seeing christina cooper . her...,SUBJECTIVE CHIEF COMPLAINT Long-standing iron ...,1352,361
78,"Doctor: Hello, patient, I understand you're he...",SUBJECTIVE CHIEF COMPLAINT Allergies. HISTORY ...,312,174
145,"doctor: Good morning, Mrs. patient. I understa...",SUBJECTIVE CHIEF COMPLAINT Depression and fati...,212,164


In [23]:
reference_sub = sub_set_test["note"].tolist()
dialogue_sub = sub_set_test["dialogue"].tolist()

In [25]:
sub_set_test = Dataset.from_pandas(sub_set_test)

In [26]:
# tokennize
max_input_length = 2048
max_output_length = 1024
def process_data_to_model_inputs(batch):
    start_prompt = 'Summarize the following patientdoctor dialogue into 2 sectios:"SUBJECTIVE CHIEF COMPLAINT" and "ASSESSMENT AND PLAN". Include all medically relevant information, including family history, diagnosis, past medical and surgical history, immunizations, lab results and known allergies.Dialogue:.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in batch["dialogue"]]
    # tokenize the inputs and labels
    inputs = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["note"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # # We have to make sure that the PAD token is ignored
    # batch["labels"] = [
    #     [-100 if token == tokenizer.pad_token_id else token for token in labels]
    #     for labels in batch["labels"]
    # ]

    return batch

In [27]:
batch_size = 2
sub_set_test = sub_set_test.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "note", "dialogue","Dialogue_Length","Note_Length"],
)

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [30]:
def model_generation(model, tokenizer, dataset):
    predictions = []

    for example in dataset:
        input_ids = example["input_ids"]
        attention_mask = example["attention_mask"]
        global_attention_mask = example["global_attention_mask"]
        reference_ids = example["labels"]

        # Decode input_ids
        input_text = tokenizer.decode(input_ids, skip_special_tokens=True)

        # Generate summary
        outputs = model.generate(input_ids=torch.tensor([input_ids]),
                                 attention_mask=torch.tensor([attention_mask]),
                                 global_attention_mask=torch.tensor([global_attention_mask]),
                                 num_beams= 4,
                                 min_length=100,
                                 max_length=512,
                                 length_penalty=2,
                                 no_repeat_ngram_size=3)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append to lists for ROUGE calculation
        predictions.append(generated_text)
    return predictions




In [31]:
# Call the function with the correct argument
with torch.cuda.amp.autocast():
    led_base_prediction_sub = model_generation(model, tokenizer, sub_set_test)

In [32]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(led_base_prediction_sub, reference_sub):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1: 0.4155886292532036
Average ROUGE-2: 0.17914399670045789
Average ROUGE-L: 0.2175679632440034


In [ ]:
data = {
    "Dialogue": dialogue_sub,
    "Reference": reference_sub,
    "Prediction": led_base_prediction_sub
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/W266_Project/Data/Results'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

df.to_csv(os.path.join(output_dir, 'LED_base_prediction_sub.csv'), index=False)